In [0]:
# from pyspark.sql import Row

# # Sample data
# data = [
#     Row(id=1, name="Raj", age=30),
#     Row(id=2, name="Virat", age=25),
#     Row(id=3, name="Mahesh", age=35)
# ]

# # Create DataFrame and temp view
# df = spark.createDataFrame(data)
# df.createOrReplaceTempView("src")

# # Show the temp view
# display(spark.sql("SELECT * FROM src"))

# # Define target table and primary key columns
# target_table = "default.sample_target"
# pk_cols = ["id"]



id name age 1 Raj 30 2 Virat 25 3 Mahesh 35

In [0]:
# # Create target table if not exists
# create_table_query = f"""
#     CREATE TABLE IF NOT EXISTS {target_table}
#     USING DELTA AS SELECT * FROM src WHERE 1 = 0
# """
# print(create_table_query)
# spark.sql(create_table_query)

# # Build ON expression for MERGE
# on_expr = " AND ".join([f"t.{c} = s.{c}" for c in pk_cols])

# print(on_expr)

# # Perform MERGE (SCD-1 upsert)
# merge_query = f"""
#     MERGE INTO {target_table} t
#     USING src s
#     ON   {on_expr}
#     WHEN MATCHED THEN UPDATE SET *
#     WHEN NOT MATCHED THEN INSERT *
# """
# print(merge_query)
# spark.sql(merge_query)

# # Show the target table after merge
# display(spark.table(target_table))


    CREATE TABLE IF NOT EXISTS default.sample_target
    USING DELTA AS SELECT * FROM src WHERE 1 = 0

t.id = s.id

    MERGE INTO default.sample_target t
    USING src s
    ON   t.id = s.id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *



id name age 3 Mahesh 35 4 Rahul 25 2 Virat 25 5 Ajay 35 1 Raj 30

In [0]:
# Databricks notebook: silver_to_gold_star_schema.py
# ==================================================
from datetime import datetime
from pyspark.sql.functions import (
    col, expr, sequence, to_date, explode, lit, date_format
)

from pyspark.sql import DataFrame

# ---------------------------------------------------------------------------
# 1. Minimal SCD-1 helper (Delta MERGE)
# ---------------------------------------------------------------------------
def upsert_scd1(src: DataFrame, target_table: str, pk_cols: list) -> None:
    """
    Overwrite-on-key (SCD-1) into a Delta table.
    If the target doesn't exist, it's created with the src schema.
    """
    # Register source DataFrame as temp view for SQL operations
    src.createOrReplaceTempView("src")

    # Create target Delta table if it does not exist
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {target_table}
        USING DELTA AS SELECT * FROM src WHERE 1 = 0
    """)

    # Build ON clause for MERGE using primary key columns
    on_expr = " AND ".join([f"t.{c} = s.{c}" for c in pk_cols])

    # Perform SCD-1 upsert using Delta MERGE
    spark.sql(f"""
        MERGE INTO {target_table} t
        USING src s
        ON   {on_expr}
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

# ---------------------------------------------------------------------------
# 2. Build / refresh DATE dimension (prod.gold.dim_date)
# ---------------------------------------------------------------------------
def refresh_dim_date(start="2000-01-01", end="2035-12-31") -> None:
    # Generate a date sequence DataFrame for the date dimension
    df = (
        spark.range(0, 1)
        .select(sequence(to_date(lit(start)), to_date(lit(end))).alias("date_seq"))
        .select(explode("date_seq").alias("Date_ID"))
        .withColumn("Year", expr("year(Date_ID)"))
        .withColumn("Month", expr("month(Date_ID)"))
        .withColumn("Day", expr("day(Date_ID)"))
        .withColumn("Week", expr("weekofyear(Date_ID)"))
        .withColumn("Quarter", expr("(month(Date_ID)-1)/3 + 1"))
        .withColumn("Weekday", date_format(col("Date_ID"), "EEEE"))
    )

    # Overwrite the dim_date table with the new date dimension data
    (df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable("prod.gold.dim_date"))

    # display(df)

# ---------------------------------------------------------------------------
# 3. Distribute silver layer to gold star-schema
# ---------------------------------------------------------------------------
# The "-> None" in the function definition indicates that the function does not return any value.
# It is a type hint specifying that the function's return type is None.
def distribute_to_gold() -> None:
    # Read cleaned transactions from silver layer
    df = spark.table("prod.silver.clean_transactions")

    # ----- Dimensions -----
    # Upsert customer dimension
    upsert_scd1(
        df.select(
            "Customer_ID", "Customer_Name", "Gender", "DOB",
            "Email", "Phone", "Customer_City"
        ).distinct(),
        "prod.gold.dim_customer",
        ["Customer_ID"]
    )

    # Upsert merchant dimension
    upsert_scd1(
        df.select(
            "Merchant_ID", "Merchant_Name",
            "Merchant_Category", "Merchant_Country"
        ).distinct(),
        "prod.gold.dim_merchant",
        ["Merchant_ID"]
    )

    # Upsert card dimension
    upsert_scd1(
        df.select(
            "Card_ID", "Card_Type", "Issuer_Bank",
            "Card_Tier", "Expiry_Date"
        ).distinct(),
        "prod.gold.dim_card",
        ["Card_ID"]
    )

    # Upsert location dimension
    upsert_scd1(
        df.select(
            "Location_ID", "City", "State", "Country"
        ).distinct(),
        "prod.gold.dim_location",
        ["Location_ID"]
    )

    # ----- Fact -----
    # Prepare fact table DataFrame with foreign keys to dimensions
    fact_df = (
        df.select(
            "Transaction_ID",
            "Transaction_Date",
            "Transaction_Amount",
            "Transaction_Status",
            "Transaction_Type",
            "Customer_ID",
            "Card_ID",
            "Merchant_ID",
            "Location_ID",
            col("Transaction_Date").alias("Transaction_Date_ID")  # FK -> dim_date
        )
    )

    # Append new fact records to the fact_transactions table
    (fact_df.write
        .format("delta")
        .mode("append")
        .saveAsTable("prod.gold.fact_transactions"))

    # display(fact_df)

# ---------------------------------------------------------------------------
# 4. Orchestration entry point
# ---------------------------------------------------------------------------
def run_star_schema_refresh() -> str:
    """
    Databricks Job entry:
      1. Refresh dim_date
      2. Push silver to gold star-schema
    """
    # Refresh the date dimension table
    refresh_dim_date()
    # Distribute silver data to gold star-schema (dimensions and fact)
    distribute_to_gold()
    return "Silver -> Gold star-schema refresh complete."

# For ad-hoc runs in a notebook:
if __name__ == "__main__":
    print(run_star_schema_refresh())

Silver -> Gold star-schema refresh complete.


In [0]:
# %sql
# select * from prod.gold.dim_customer

Customer_ID Customer_Name Gender DOB Email Phone Customer_City C1270 Vickieprice OTHER 1948-04-11 vickie.price@outlook.com 9050043433 Ahmedabad C1342 Kennethhowarddds OTHER 1947-12-08 kenneth.howard.dds@outlook.com 8460106611 Kolkata C1506 Timothybishopjr OTHER 1974-06-15 timothy.bishop.jr.@yahoo.com 8753645397 Lucknow C1825 Alexanderhanna MALE 1992-05-18 null 6799688028 Kochi C1745 Graceedwards MALE 1967-05-18 grace.edwards@outlook.com 9290129421 Ahmedabad C1981 Patricknelson OTHER 1970-03-09 patrick.nelson@rediffmail.com 9836183666 Bangalore C1916 Terryestes MALE 1988-09-25 terry.estes@gmail.com 6714297307 Chandigarh C1983 Dalebaker OTHER 1946-01-25 dale.baker@rediffmail.com 9906725691 Ahmedabad C1347 Sherryfigueroa FEMALE 1958-06-18 sherry.figueroa@outlook.com 7749974999 Kochi C1612 Kristinealvarado MALE 1955-12-04 kristine.alvarado@rediffmail.com 9383046934 Kolkata C1628 Adammccoy MALE 2001-03-20 adam.mccoy@rediffmail.com 8296153530 Kolkata C1301 Mrdonaldnguyen MALE 1964-10-13 mr..donald.nguyen@yahoo.com 6142362794 Chandigarh C1126 Courtneyhayes OTHER 1974-03-30 courtney.hayes@yahoo.com 7470664930 Delhi C1586 Dawnburke FEMALE 2000-07-11 dawn.burke@yahoo.com 8435474756 Bangalore C1209 Donaldevans MALE 1954-05-28 donald.evans@rediffmail.com 6971052481 Hyderabad C1559 Karencollins MALE 2000-03-12 karen.collins@gmail.com 6461778595 Kochi C1684 Larrycarter FEMALE 1954-05-20 larry.carter@outlook.com 9642626596 Delhi C1538 Loricarter FEMALE 2004-01-07 lori.carter@outlook.com 7863330883 Ahmedabad C1231 Laurapatel MALE 1999-11-01 laura.patel@yahoo.com 7587569877 Bangalore C1640 Jessicacarrillo FEMALE 1979-08-07 jessica.carrillo@yahoo.com 8966050447 Lucknow C1101 Jenniferrusso OTHER 1982-09-11 jennifer.russo@rediffmail.com 9446510099 Mumbai C1142 null FEMALE 2006-11-03 null 7841579769 Bangalore C1615 Scottsolis MALE 1994-04-03 scott.solis@yahoo.com 7894243805 Bangalore C1850 Kathrynstewart FEMALE 2005-06-24 kathryn.stewart@outlook.com 9529984898 Ahmedabad C1398 Alexanderluna MALE 1972-08-23 alexander.luna@outlook.com 6961287099 Hyderabad C1709 Williamgriffin FEMALE 1975-09-15 william.griffin@yahoo.com 9682847568 Ahmedabad C1298 Mariabyrd OTHER 1983-06-01 maria.byrd@gmail.com null Mumbai C1905 Meghanmorris FEMALE 1965-08-14 meghan.morris@outlook.com 8944808215 Lucknow C1747 Nicolewright MALE 1980-07-31 nicole.wright@rediffmail.com 6357399985 Bangalore C1930 Benjaminbutler FEMALE 1982-11-04 benjamin.butler@gmail.com 6778591674 Delhi C1817 Williamwalkerdvm FEMALE 1991-03-17 william.walker.dvm@rediffmail.com 9427279058 Mumbai C1696 Taylorbrooks MALE 2004-12-12 taylor.brooks@yahoo.com null Mumbai C1925 Lukehansen OTHER 1976-09-16 luke.hansen@outlook.com 7704297756 Hyderabad C1546 Anthonydavidson FEMALE 1974-10-27 anthony.davidson@rediffmail.com 6551149736 Chandigarh C1674 Terriwilliams MALE 1944-08-09 terri.williams@gmail.com 9788244543 Bangalore C1758 Michaelbennettdvm MALE 1987-06-02 michael.bennett.dvm@gmail.com 8002982740 Mumbai C1359 Jonathandavis MALE 2005-02-18 jonathan.davis@outlook.com 8484992187 Delhi C1645 Caitlinjohnson OTHER 1979-12-13 caitlin.johnson@rediffmail.com 7193548892 Hyderabad C1892 Dwaynecarlson OTHER 1997-12-10 dwayne.carlson@rediffmail.com 8632408610 Chandigarh C1661 Stevenscott OTHER 1951-03-12 steven.scott@gmail.com 6475235053 Mumbai C1704 Troyrivera OTHER 1946-08-31 troy.rivera@yahoo.com 6058280611 Kolkata C1746 Aprilharrell MALE 2003-11-20 april.harrell@rediffmail.com 8794560575 Kochi C1583 Kimberlyibarra OTHER 1979-07-18 kimberly.ibarra@yahoo.com 8392944942 Chennai C1176 Williambrooks OTHER 1977-01-05 william.brooks@gmail.com 6593328086 Chennai C1279 Ashleymendoza FEMALE 1970-03-30 ashley.mendoza@yahoo.com 8169652549 Lucknow C1669 Heatherhess MALE 1946-07-31 heather.hess@yahoo.com 8991305222 Hyderabad C1998 Johnturner FEMALE 1979-04-18 john.turner@yahoo.com 7061880146 Lucknow C1549 Tracycoleman OTHER 1964-02-12 tracy.coleman@outlook.com 6602719859 Delhi C1223 Hannahbruce OTHER 1970-12-02 hannah.bruce@yahoo.